In [1]:
import requests
import os
from dotenv import load_dotenv
from pathlib import Path
import json
import pandas as pd
# import psycopg2

In [2]:
def load_secrets():
    load_dotenv()
    env_path = Path(".") / ".env"
    load_dotenv(dotenv_path=env_path)

    API_KEY = os.getenv("API_KEY")
    return API_KEY

In [3]:
API_KEY = load_secrets()

In [4]:
url_recent = f"https://airquality.googleapis.com/v1/currentConditions:lookup?key={API_KEY}"

In [5]:
headers = {"Content-Type": "application/json"}

In [6]:
CAPITALS = {
    "Rio Branco": (-9.97499, -67.8243),
    "Maceió": (-9.66599, -35.735),
    "Macapá": (0.0389, -51.0664),
    "Manaus": (-3.11866, -60.0212),
    "Salvador": (-12.9714, -38.5014),
    "Fortaleza": (-3.73186, -38.5267),
    "Brasília": (-15.7797, -47.9297),
    "Vitória": (-20.3155, -40.3128),
    "Goiânia": (-16.6786, -49.2539),
    "São Luís": (-2.52972, -44.3028),
    "Cuiabá": (-15.5989, -56.0949),
    "Campo Grande": (-20.4428, -54.6464),
    "Belo Horizonte": (-19.9167, -43.9345),
    "Belém": (-1.45502, -48.5024),
    "João Pessoa": (-7.115, -34.8631),
    "Curitiba": (-25.4278, -49.2731),
    "Recife": (-8.05, -34.9),
    "Teresina": (-5.08921, -42.8016),
    "Rio de Janeiro": (-22.9068, -43.1729),
    "Natal": (-5.795, -35.2094),
    "Porto Alegre": (-30.0331, -51.23),
    "Porto Velho": (-8.76194, -63.9039),
    "Boa Vista": (2.81972, -60.6733),
    "Florianópolis": (-27.5969, -48.5495),
    "São Paulo": (-23.5505, -46.6333),
    "Aracaju": (-10.9111, -37.0717),
    "Palmas": (-10.1675, -48.3277)
}

In [7]:
df = pd.DataFrame()

In [8]:
capitals_dict = {}
for capital, coords in CAPITALS.items():
    response = requests.post(url=url_recent,
                             json={"location": {
                                 "latitude": coords[0], 
                                 "longitude": coords[1]
                                 }
                            },
                             headers=headers)
    capitals_dict.update({capital : response.text})

In [9]:
print(response.text)

{
  "dateTime": "2025-09-25T19:00:00Z",
  "regionCode": "br",
  "indexes": [
    {
      "code": "uaqi",
      "displayName": "Universal AQI",
      "aqi": 62,
      "aqiDisplay": "62",
      "color": {
        "red": 0.70980394,
        "green": 0.8862745,
        "blue": 0.11764706
      },
      "category": "Good air quality",
      "dominantPollutant": "o3"
    }
  ]
}



In [10]:
rows = []
for city, data_json in capitals_dict.items():
    city_data = json.loads(data_json)
    idx = city_data.get('indexes')[0]
    rows.append({
        "City": city,
        "Date/Time": city_data.get("dateTime"),
        "Local" : city_data.get("regionCode"),
        "AQI": idx["aqi"],
        "Red": idx.get("color",{}).get("red"),
        "Green": idx.get("color",{}).get("green"),
        "Blue": idx.get("color",{}).get("blue"),
        "Label": idx.get("category"),
        "Dominant Pollutant": idx.get("dominantPollutant")
    })
city_data

{'dateTime': '2025-09-25T19:00:00Z',
 'regionCode': 'br',
 'indexes': [{'code': 'uaqi',
   'displayName': 'Universal AQI',
   'aqi': 62,
   'aqiDisplay': '62',
   'color': {'red': 0.70980394, 'green': 0.8862745, 'blue': 0.11764706},
   'category': 'Good air quality',
   'dominantPollutant': 'o3'}]}

In [11]:
df = pd.concat([df, pd.DataFrame(rows)], ignore_index=True)

In [12]:
df

,City,Date/Time,Local,AQI,Red,Green,Blue,Label,Dominant Pollutant
0,Rio Branco,2025-09-25T19:00:00Z,br,70,0.517647,0.811765,0.200000,Good air quality,o3
1,Maceió,2025-09-25T19:00:00Z,br,78,0.376471,0.756863,0.203922,Good air quality,pm25
2,Macapá,2025-09-25T19:00:00Z,br,74,0.447059,0.784314,0.200000,Good air quality,o3
3,Manaus,2025-09-25T19:00:00Z,br,67,0.588235,0.839216,0.168627,Good air quality,o3
4,Salvador,2025-09-25T19:00:00Z,br,66,0.611765,0.847059,0.156863,Good air quality,pm25
5,Fortaleza,2025-09-25T19:00:00Z,br,66,0.611765,0.847059,0.156863,Good air quality,o3
6,Brasília,2025-09-25T19:00:00Z,br,65,0.635294,0.858824,0.149020,Good air quality,o3
7,Vitória,2025-09-25T19:00:00Z,br,89,0.188235,0.686275,0.215686,Excellent air quality,pm10
8,Goiânia,2025-09-25T19:00:00Z,br,63,0.686275,0.874510,0.129412,Good air quality,o3
9,São Luís,2025-09-25T19:00:00Z,br,58,0.803922,0.921569,0.078431,Moderate air quality,pm10


In [13]:
url_history = f"https://airquality.googleapis.com/v1/history:lookup?key={API_KEY}"

capitals_dict_history = {}
for capital, coords in CAPITALS.items():
    response = requests.post(url=url_history,
                             json={
                                 "dateTime": "2025-09-21T15:01:23Z",
                                 "location": {
                                     "latitude": coords[0],
                                     "longitude": coords[1]
                                 },
                            },
                             headers=headers)
    capitals_dict_history.update({capital: response.text})

In [14]:
rows_history = []
for city, data_json in capitals_dict_history.items():
    capitals_dict_history = json.loads(data_json)

    idx = capitals_dict_history.get("hoursInfo")[0].get("indexes")[0]

    rows_history.append({
        "City": city,
        "Date/Time": capitals_dict_history.get("hoursInfo")[0].get("dateTime"),
        "Local" : capitals_dict_history.get("regionCode"),
        "AQI": idx.get("aqi"),
        "Red": idx.get("color",{}).get("red"),
        "Green": idx.get("color",{}).get("green"),
        "Blue": idx.get("color",{}).get("blue"),
        "Label": idx.get("category"),
        "Dominant Pollutant": idx.get("dominantPollutant")
    })

In [15]:
pd.set_option("display.max_rows", None)
df = pd.concat([df, pd.DataFrame(rows_history)], ignore_index=True)
df = df.sort_values(by="City")
df

,City,Date/Time,Local,AQI,Red,Green,Blue,Label,Dominant Pollutant
52,Aracaju,2025-09-21T15:00:00Z,br,70,0.517647,0.811765,0.200000,Good air quality,o3
25,Aracaju,2025-09-25T19:00:00Z,br,74,0.447059,0.784314,0.200000,Good air quality,o3
39,Belo Horizonte,2025-09-21T15:00:00Z,br,70,0.517647,0.811765,0.200000,Good air quality,pm10
12,Belo Horizonte,2025-09-25T19:00:00Z,br,62,0.709804,0.886274,0.117647,Good air quality,pm25
40,Belém,2025-09-21T15:00:00Z,br,64,0.658824,0.866667,0.137255,Good air quality,o3
13,Belém,2025-09-25T19:00:00Z,br,66,0.611765,0.847059,0.156863,Good air quality,o3
49,Boa Vista,2025-09-21T15:00:00Z,br,78,0.376471,0.756863,0.203922,Good air quality,pm10
22,Boa Vista,2025-09-25T19:00:00Z,br,72,0.482353,0.796078,0.200000,Good air quality,o3
33,Brasília,2025-09-21T15:00:00Z,br,59,NaN,NaN,NaN,Moderate air quality,pm10
6,Brasília,2025-09-25T19:00:00Z,br,65,0.635294,0.858824,0.149020,Good air quality,o3


In [16]:
# def preprocess_air_quality():
#     conn = psycopg2.connect(
#         host='Localhost',
#         dbname="meubanco",
#     )sdavadv